In [14]:
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [5]:
import skimage
from scipy import ndimage 

In [13]:
def show_image_list(img_list, nc=2, cmap='gray', titlez=None):    
    n = len(img_list)
    nr = n//nc + ( 0 if n%nc == 0 else 1) 
    for i, img in enumerate(img_list):
        plt.subplot(nr, nc, (i+1) )
        plt.imshow( img, cmap=cmap)
        plt.axis('off')
        if titlez:
            plt.title( f"{titlez[min(i, len(titlez)-1)]}" )
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.show();
     
def show_image(img, cmap='gray', binz=None):
        plt.subplot(1,2,1)
        if (cmap is None):
            plt.imshow(img)
        else:
            plt.imshow(img, cmap=cmap) 
        plt.axis('off')
        plt.subplot(1,2,2)
        plt.hist(img.flatten()*(1/img.max()), bins=binz)
        plt.title(f"Histogram: {binz} bins")
        plt.tick_params(axis='y', which='both', labelleft=False, labelright=True)
        plt.subplots_adjust(wspace=0, hspace=0)
        

----

In [7]:
## Prioritize ndimage operations over skimage or the best of the two??
class AnImage:        
    def __init__(self, src, isgray=True):
        self.img = skimage.io.imread( src )
        if isgray:
            self.img = skimage.color.rgb2gray( self.img )
    
    @property
    def rgb(self):
        return skimage.color.gray2rgb(self.img)
    
    @property
    def gray(self):
        return skimage.color.gray2rgb(self.img)
    
    def show(self, cmap='gray', binz=None):
        plt.subplot(1,2,1)
        if (cmap is None):
            plt.imshow(self.img)
        else:
            plt.imshow(self.img, cmap=cmap) 
        plt.axis('off')
        plt.subplot(1,2,2)
        plt.hist(self.img.flatten()*(1/self.img.max()), bins=binz)
        plt.title(f"Histogram: {binz} bins")
        plt.tick_params(axis='y', which='both', labelleft=False, labelright=True)
        plt.subplots_adjust(wspace=0, hspace=0)
        
    def contour(self):
        plt.contour(self.img) ##TODO: more at levels 
        
    @property
    def noisy(self):
        outi = self.img.copy()
        outi = outi + 0.4 * outi.std() * np.random.random(outi.shape)
        return outi
    
    @property
    def stats(self): 
        return f"---Image Stats---\n \
Shape: {self.img.shape} \n \
Type: { type( self.img )} \n \
Mean: { np.mean( self.img )} \n \
Median: {np.median( self.img )} \n \
Max: {np.max( self.img )} \n \
Min: {np.min( self.img )} "
    
    def denoised(self, sig=3):
        # Gaus will smooth but at expense of edges so keep sig small
        return ndimage.gaussian_filter(self.img, sig)
    
    def colorised(self, r=100, g=20, b=0):
        ## Luma transform weights 
        # LR = wR + xG + yB
        # (w,x,y) = (299, 587, 114)/1000
#         colored =  skimage.img_as_float(self.make_3d() )
#         Ltrans = np.array([299, 587, 114])*(1/1000)
#         R, G, B = colored[:,:,0], colored[:,:,1], colored[:,:,2]
#         colored = (R*700 + G*587 + B*114)/1000

        colored = self.make_3d()
        colored[:,:,0] = r
#         colored[:,:,1] = g
#         colored[:,:,2] = b
        return colored
            
    
    def make_3d(self):
        if len(self.img.shape) >= 3:
            return self.img.copy()
        
        nr, nc = self.img.shape
        f = np.empty( (nr, nc, 3))
        ##brute force
        for i in range(nr):
            for j in range(nc):
                for k in range(3):
                    f[i,j,k] = img.img[i,j]*(i*j)
        return f
    
class ArrayImage(AnImage):    
    def __init__(self, matrix):
        self.img = matrix

In [8]:
   
def make_gaussian_kern(size, sig=1):    
    ## size better if odd b/c balance
    ## The larger the size of the kernel, more blur <<< the lower the sensitivity to noise << 
    outk = np.zeros( (size, size) )
    
    k = (size-1)//2
    sig_const = 1/( 2 * np.pi * sig) 
    
    for i in range(size):
        for j in range(size):
            outk[i,j] = sig_const * np.exp( ( ( (i+1)-(k+1))**2 +( (j+1)-(k+1))**2 )*(-1/(2*sig)))
    return outk

In [9]:

class Junkie:
    @staticmethod 
    def equalize_histogram(img, binz=20, method=1):
        p2, p98 = np.percentile(img, (2,98))
        method_str = ['rescale', 'eq_hist', 'adapt_hist', '']
        method_fx = [
            (skimage.exposure.rescale_intensity, {'in_range':(p2,p98)}),
            (skimage.exposure.equalize_hist, {}),
            (skimage.exposure.equalize_adapthist, {'clip_limit':0.03}),
        ]
        
        plt.title(f"{method_str[method]}")
        plt.subplot(2,2,1)
        plt.imshow(img, cmap='gray')
        plt.axis('off')

        plt.subplot(2,2,2)
        plt.hist(img.flatten(), bins=binz, color='b')
        cdf, cbinz = skimage.exposure.cumulative_distribution(img, binz)
        plt.plot(cbinz, cdf, 'r')
        plt.tick_params(axis='y', which='both', labelleft=False, labelright=True)

        fx = method_fx[method]
        img_eq = fx[0](img, **fx[1])

        plt.subplot(2,2,3)
        plt.imshow(img_eq, cmap='gray')
        plt.axis('off')

        plt.subplot(2,2,4)
        plt.hist(img_eq.flatten(), bins=binz, color='b')
        cdf, cbinz = skimage.exposure.cumulative_distribution(img_eq, binz)
        plt.plot(cbinz, cdf, 'r')
        plt.tick_params(axis='y', which='both', labelleft=False, labelright=True)

        return img_eq
    
    @staticmethod
    def equlization_flow(fpath, nc=2):
        img = AnImage(fpath)

        print('Default Equalize')
        eq1 = Junkie.equalize_histogram(img.img, method=1)
        plt.show()
        plt.clf();

        print('Adaptive Equalize')
        eq2 = Junkie.equalize_histogram(img.img, method=2)
        plt.show()
        plt.clf();

        print('Percentile Rescaling - Contrast Stretching')
        eq0 = Junkie.equalize_histogram(img.img, method=0)
        plt.show()
        plt.clf();
        
        show_image_list([img.img, eq1, eq2, eq0], titlez=['origi', 'default equi', 'adapt-hist', 'percentile'], nc=nc)
        plt.show(); 

In [4]:
import abc

In [11]:
class AFilter:
    @staticmethod 
    def pad(img, kern):
        w, h = img.shape 
        kw, kh = kern.shape
        pw, ph = (kw - 1)//2, (kh-1)//2
        
        pot = np.zeros( (w+pw*2, h+ph*2) )
        pot[ pw:-pw, ph:-ph] = img 
        
        return pot
    
    @staticmethod 
    def get_kern(k):
        ## flip 180 
        flipped = np.flip(k, 1)
        return flipped
    
    @staticmethod 
    def get_convolve_value(k, phood):
        ## sum of element-wise products
        return np.sum( k * phood)
        
#     @abc.abstractmethod
#     def apply(img, kern, padit=True):
#         NotImplementedError
        
    @staticmethod 
    def convolve(img, kern, padit=True):         
        flipped_kern = AFilter.get_kern(kern)
                
        in_img = AFilter.pad(img, kern) if padit else img.copy() 
        output = np.zeros(in_img.shape)
        
        rowz, colz = in_img.shape 
        kw, kh = kern.shape
        
        ##TODO: fix padding/border-crossing
        for i in range(1, rowz+1):
            for j in range(1, colz+1):
                phood = in_img[ i:i+kw, j:j+kh ]
                output[ i, j] = AFilter.get_convolve_value(flipped_kern, phood)
                
                if (j+kh)>= colz:
                    break
            if (i+kw) >= rowz:
                break
        return output
    
    
class BaseFilter(AFilter):
    @staticmethod
    def apply(img, kern, padit=True):
        return AFilter.convolve(img, kern, padit)
        ## using numpy convolve << BUT is 1-D convolve
        ## return np.convolve(img, kern)
    